
[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip


In [12]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
PATH_bva = "data/annotations/20180415_bva.json"
PATH_cyber = "data/annotations/20180415_cyber_crime.json"
PATH_intellectual = "data/annotations/20180415_intellectual_property.json"
PATH_scotus = "data/annotations/20180415_scotus.json"

PATHS = [PATH_bva, PATH_cyber, PATH_intellectual, PATH_scotus]
test_PATHS = [PATH_cyber]

In [14]:

for path_ in test_PATHS:
    dataset = json.load(open(path_, "r", encoding="utf8"))
    for doc in dataset:
        x = dataset[doc]['text']
        y = []
        for sentence in dataset[doc]['annotations']:
            print(sentence['start'],sentence['end'])
            y.append(dataset[doc]['text'][sentence['start']:sentence['end']])

0 57
58 96
97 123
124 157
158 182
183 206
360 373
374 420
422 460
461 485
486 491
492 585
587 730
1071 1239
1240 1567
1568 1735
1737 1780
1782 1974
1975 2104
2105 2249
2250 2390
2391 2525
2526 2590
2591 2741
2743 2853
2854 2935
2936 3107
3108 3285
3286 3336
3338 3564
3565 3674
3675 3873
3874 4015
4299 4435
4436 4635
4637 4639
4640 4845
4846 5180
5181 5186
5187 5192
5193 5195
5196 5366
5367 5497
5499 5717
5718 5833
5834 5853
5929 5975
5977 5979
6083 6143
6144 6333
6335 6460
6461 6520
6521 6730
6731 6758
6759 6894
6895 6898
6900 7080
7081 7096
7097 7260
7261 7281
7784 7786
7787 7822
7824 7940
7941 8072
8073 8277
8279 8468
8557 8765
8766 8867
8869 8982
9420 9591
9593 9732
9733 9828
9829 9849
9850 10005
10006 10169
10170 10392
10393 10709
10928 11130
11132 11485
11486 11566
11568 11570
11571 11601
11603 11753
11754 11777
11778 11874
11875 12022
12024 12130
12131 12341
12342 12580
12581 12641
12643 12902
12903 13048
13050 13235
13236 13542
13544 13576
13578 13783
13784 14137
14312 14393
143

In [15]:
def TF_PN(original, pred):
    original = original.replace("\n"," ").split()
    pred = pred.split()

    # print(original)
    # print(pred)
    
    beggining = [0, 0, 0, 0] #TP, TN, FP, FN
    end = [0, 0, 0, 0]
    i_max = [0, 0, 0, 0]

    if original[0] == pred[0]:
        beggining[0]+=1
        i_max[1]+=1
        end[1]+=1

    else:
        beggining[2]+=1
        i_max[3]+=1

    if original[-1] == pred[-1]:
        end[0]+=1
        i_max[1]+=1
        beggining[1]+=1

    else:
        end[2]+=1
        i_max[3]+=1

    change = False
    for pos in range(len(original)):
        # print(i_max[0], len(pred))
        if i_max[0] == len(pred):
            beggining[3] =0
            break
        
        if original[pos] != pred[0]:
            continue

        else:
            change = True
        
        i = [0, 0, 0, 0] #TP, TN, FP, FN
        temp_beg = beggining
        temp_end = end

        tip = len(original) if original < pred else pred

        for nth, (x, y) in enumerate(zip(range(pos, len(original)), range(len(pred)))):
            if original[x]==pred[y]:
                i[0]+=1
                temp_beg[1]+=1
                temp_end[1]+=1

            else:
                i[2]+=1
                temp_beg[1]+=1
                if nth!=tip:
                    temp_end[3]+=1


        if i[0]>i_max[0]:
            i_max = i
            if temp_beg[3]==0 and pos!=0:
                temp_beg[3]+=1

        elif i[0]==i_max[0] and (i[2] == 0) :
            i_max = i


            beggining = temp_beg
            end = temp_end
        

    if not change:
        i_max[2] = len(pred)-2

    return beggining, i_max, len(original), len(pred),  end


In [6]:
cont_table = {}

In [16]:
def check_range(pointer_start, pointer_end, ranges):
    intersection_dict = {}
    for i in ranges:
        if pointer_start>=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end>=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [i['end'] - i['start'], abs(i['start']-pointer_start)]
        
        if pointer_start>=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - pointer_start, abs(i['start']-pointer_start)]

        if pointer_start<=i['start'] and pointer_end<=i['end']:
            intersection_dict[str(i['start'])+'-'+str(i['end'])] = [pointer_end - i['start'], abs(i['start']-pointer_start)]

    max_intersection_key = None
    max_intersection_len = -1
    max_intersection_start_dist = 1000000
    
    for i in intersection_dict.keys():
        if intersection_dict[i][0]>max_intersection_len or (intersection_dict[i][0]==max_intersection_len and max_intersection_start_dist<intersection_dict[i][1]):
            max_intersection_len =  intersection_dict[i][0]
            max_intersection_key = i
            max_intersection_start_dist = intersection_dict[i][1]

    return max_intersection_key

In [17]:
results = {}

In [18]:
def accuracy(x):
    if sum(x) == 0:
        return 0
    return (x[0]+x[1])/sum(x)

def recall(x):  #TP, TN, FP, FN
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[3])

def precision(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0]+x[2])

def f1_score(x):
    if x[0] == 0:
        return 0

    return x[0]/(x[0] + 0.5*(x[2]+x[3]))

def specificity(x):
    if x[1] == 0:
        return 0

    return x[1]/(x[1] + x[2])

def acc_recall_precision_f1(x):
    acc, rec, prec, f1, specificity_  = 0, 0, 0, 0, 0
    t = 0
    for i in x:
        acc = acc + accuracy(i)
        rec = rec + recall(i)
        prec = prec + precision(i)
        f1 = f1 + f1_score(i)
        specificity_ = specificity_ + specificity(i)
        

    return acc/len(x), rec/len(x), prec/len(x), f1/len(x), specificity_/len(x)


# for paths in results:
#     print(paths)
#     B = []
#     I = []
#     L = []

#     for i in results[paths]:
#         B.append(i[0])
#         I.append(i[1])
#         L.append(i[-1])


#     print(f"B : {acc_recall_precision_f1(B)}")
#     print(f"I : {acc_recall_precision_f1(I)}")
#     print(f"L : {acc_recall_precision_f1(L)}\n")

In [12]:
pip install flask

  Using cached Flask-2.2.3-py3-none-any.whl (101 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached Werkzeug-2.2.3-py3-none-any.whl (233 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\sneha\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [13]:
pip install -U flask-cors

  Using cached Flask_Cors-3.0.10-py2.py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sneha\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [19]:
import flask
import flask_cors

In [15]:
pip install python-crfsuite


     ------------------------------------ 139.4/139.4 KB 516.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sneha\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [20]:
from luima_sbd import *
import argparse
import codecs
import luima_sbd.config as config
import luima_sbd.server_utils as server_utils
import sys
import luima_sbd.sbd_utils as sbd_utils



for sent in sbd_utils.text2sentences(dataset[doc]['text']):
    print(sent)
    



Wenger v. State, 292 S.W.3d 191 (Tex. App. 2009)
Court of Appeals of Texas
Filed: June 25th, 2009
Precedential Status: Precedential
Citations: 292 S.W.3d 191
Docket Number: 2-07-282-CR
Author: Lee Ann Dauphinot
292 S.W.3d 191 (2009)
Ruben WENGER V a/k/a Reuben V. Wenger, Appellant
v.
The STATE of Texas, State.
No. 2-07-282-CR.
Court of Appeals of Texas, Fort Worth.
June 25, 2009.
*193
Lisa Mullen, Fort Worth, for Appellant.
Joe Shannon, Jr., Criminal District Attorney, Charles M. Mallin, Chief, Appellate Division, and Debra Ann Windsor, Marty Purselley and C. Page Simpson, Assistant District Attorneys for Tarrant County, Fort Worth, TX, for Appellee.
Panel: DAUPHINOT, GARDNER, and MEIER, JJ.
OPINION
ANNE GARDNER, Justice.
I.
Introduction
Appellant Ruben Wenger appeals his conviction on three counts of promotion of child pornography and four counts of possession of child pornography as alleged in the same indictment.
Appellant argues that the evidence is legally and factually insufficie

In [21]:
import pickle 
import os

In [22]:
def CRF_model(text):
    return sbd_utils.text2sentences(text)

results = {}
docnum=0;

y_full = []
p_full =[]

for path_ in test_PATHS:
    dataset = json.load(open(path_, "r", encoding="utf8"))
    results[path_] = []
    for doc in dataset:
        docnum = docnum+1
        x = dataset[doc]['text']
        y = []
        for sentence in dataset[doc]['annotations']:
            y.append(dataset[doc]['text'][sentence['start']:sentence['end']])
        y_full.append(y)
       
        pred = CRF_model(x.replace("\r","\n").replace("\t","\n"))
        
        p = []

        for pred_sentences in pred:
            p.append(pred_sentences)
        
        p_full.append(p)

In [23]:
with open('ground_truth.pickle', 'wb') as handle:
    pickle.dump(y_full, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('pred_crf.pickle', 'wb') as handle:
    pickle.dump(p_full, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
def CRF_model(text):
    return sbd_utils.text2sentences(text)

results = {}
docnum=0;

for path_ in test_PATHS:
    dataset = json.load(open(path_, "r", encoding="utf8"))
    results[path_] = []
    for doc in dataset:
        docnum = docnum+1
        x = dataset[doc]['text']
        y = []
        for sentence in dataset[doc]['annotations']:
            y.append(dataset[doc]['text'][sentence['start']:sentence['end']])
       
        pred = CRF_model(x.replace("\r","\n").replace("\t","\n"))
        # print(len(y), len(pred))
    
        pointer = 0
        mappings = []
        filtred_pred = []
        # org_pred=[]
        # no_maped = 0;

        for pred_sentences in pred:

            padding = 0
            pointer = pointer + dataset[doc]['text'][pointer:pointer+len(pred_sentences)+10].find(pred_sentences[:4])
            ending = dataset[doc]['text'][pointer:pointer+len(pred_sentences)+10].find(pred_sentences[-4:])
            maped = check_range(pointer, pointer+ending+4, dataset[doc]['annotations'])
           
            pointer += ending+4
    
            pred_sentences = pred_sentences.replace("\n"," ").replace("\r"," ").replace("\t"," ")
    
            if maped is not None :
                mappings.append(dataset[doc]['text'][int(maped.split('-')[0]):int(maped.split('-')[1])].replace("\n"," ").replace("\r"," ").replace("\t"," "))
                filtred_pred.append(pred_sentences)
                results[path_].append(TF_PN(mappings[-1], filtred_pred[-1]))
               
        # print(docnum)
        # for i in range(len(mappings)):
        #     if (mappings[i])!= (filtred_pred[i]):
                # print((mappings[i])+'\n'+(filtred_pred[i]), end='\n\n')
                # print("MP <ATHC")
                

# print(cont_Table)
with open('filtered_pred_crf.pickle', 'wb') as handle:
    pickle.dump(filtred_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

                # print(filtred_pred[-1], mappings[-1], sep = " ======= ")
                # print(results[path_][-1])


# pickle.dump(results, open("results/crf.pkl", "wb"))

In [55]:
len(cont_Table)

5963

In [ ]:
count=0
for _, __ in zip(y, pred):
    if(_!= __):
        print(_, __, sep = " === ")
        print()
        count = count+1

In [34]:
count

302

In [27]:
for paths in results:
    print(paths)
    B = []
    I = []
    L = []

    for i in results[paths]:
        B.append(i[0])
        I.append(i[1])
        L.append(i[-1])


    print(f"B : {acc_recall_precision_f1(B)}")
    print(f"I : {acc_recall_precision_f1(I)}")
    print(f"L : {acc_recall_precision_f1(L)}\n")

data/annotations/20180415_cyber_crime.json
B : (0.9526388730581733, 0.9401748040988547, 0.9401748040988547, 0.9401748040988547, 0.9527621674852652)
I : (0.94830150562476, 0.951627486437613, 0.9476234065192082, 0.9481591405911785, 0.0010174298581581691)
L : (0.9505372544801073, 0.9334157515290727, 0.9338456901748041, 0.9334356292915665, 0.948490736886739)



In [28]:
B = []
I = []
L = []

for paths in results:
    for i in results[paths]:
        B.append(i[0])
        I.append(i[1])
        L.append(i[-1])


print(f"B : {acc_recall_precision_f1(B)}")
print(f"I : {acc_recall_precision_f1(I)}")
print(f"L : {acc_recall_precision_f1(L)}\n")

B : (0.9526388730581733, 0.9401748040988547, 0.9401748040988547, 0.9401748040988547, 0.9527621674852652)
I : (0.94830150562476, 0.951627486437613, 0.9476234065192082, 0.9481591405911785, 0.0010174298581581691)
L : (0.9505372544801073, 0.9334157515290727, 0.9338456901748041, 0.9334356292915665, 0.948490736886739)



In [37]:
dataset.keys()

dict_keys(['58c1d5b70bf16802d61e59e2', '58b09c410bf16802d61e3a5c', '58ae5cd90bf16802d61e38fd', '59e7f65674b84c4d877aa805', '58b765f10bf16802d61e43d7', '58b09dca0bf16802d61e3a5d', '59e801b674b84c4d877aa813', '59e8035274b84c4d877aa818', '59e8042874b84c4d877aa819', '59e8053374b84c4d877aa81a', '58c1d58c0bf16802d61e59e0', '58b72f7e0bf16802d61e422d', '59e80a8b74b84c4d877aa81e', '59e7f54274b84c4d877aa802', '58b9e1d50bf16802d61e4716', '59e7f6fc74b84c4d877aa808', '59eb5ccb3a6b3d4ef371cf60', '59e80a2a74b84c4d877aa81b', '59e80a5d74b84c4d877aa81c'])

In [29]:
results = {}

from cgi import test
import time
total = 0

def CRF_model(text):
    return sbd_utils.text2sentences(text)

results = {}

for path_ in test_PATHS:
    dataset = json.load(open(path_, "r", encoding="utf8"))
    results[path_] = []
    
    for doc in dataset:
        x = dataset[doc]['text']
        y = []
        for sentence in dataset[doc]['annotations']:
            y.append(dataset[doc]['text'][sentence['start']:sentence['end']])
        
        start = time.time()
        pred = CRF_model(x.replace("\r","\n").replace("\t","\n"))
        total = total + time.time() - start
        # print(len(y), len(pred))
    
    
        pointer = 0
        mappings = []
        filtred_pred = []
    
        for pred_sentences in pred:
            padding = 0
            pointer = pointer + dataset[doc]['text'][pointer:pointer+len(pred_sentences)+10].find(pred_sentences[:4])
            ending = dataset[doc]['text'][pointer:pointer+len(pred_sentences)+10].find(pred_sentences[-4:])
            maped = check_range(pointer, pointer+ending+4, dataset[doc]['annotations'])
            # if maped != None:
            #     print(raw_str(pred_sentences), raw_str(dataset['59bd4ac35116540935ee6851']['text'][int(maped.split('-')[0]):int(maped.split('-')[1])]), sep=" ====== ", end="\n\n")
    
            pointer += ending+4
            # print(dataset[doc]['text'][pointer])
    
            pred_sentences = pred_sentences.replace("\n"," ").replace("\r"," ").replace("\t"," ")
    
            if maped is not None :
                mappings.append(dataset[doc]['text'][int(maped.split('-')[0]):int(maped.split('-')[1])].replace("\n"," ").replace("\r"," ").replace("\t"," "))
                filtred_pred.append(pred_sentences)
                results[path_].append(TF_PN(mappings[-1], filtred_pred[-1]))

                # print(filtred_pred[-1], mappings[-1], sep = " ======= ")
                # print(results[path_][-1])


print("CRF: ",total)

CRF:  20.02577781677246
